<a href="https://colab.research.google.com/github/webbigdata-jp/python_sample/blob/main/weblab_10b_instruction_sft_GPTQ_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# weblab-10b-instruction-sft-GPTQ

original model [weblab-10b-instruction-sft](https://huggingface.co/matsuo-lab/weblab-10b-instruction-sft) which is a Japanese-centric multilingual GPT-NeoX model of 10 billion parameters.

[weblab-10b-instruction-sft-GPTQ](https://huggingface.co/dahara1/weblab-10b-instruction-sft-GPTQ) is 4bit GPTQ Version.  

It's mean that size is smaller and the execution speed is faster, but the inference performance may be a little worse.  You need autoGPTQ library to use this model.  

In [ ]:
!pip install git+https://github.com/PanQiWei/AutoGPTQ@eea67b7e130b92605f5ace33cc93e8b95e9c12a5
# The development speed of AutoGPTQ is fast. so sometimes this model doesn't work some version.
# bug example: No response or repeat same sentence many times, etc.

  Cloning https://github.com/PanQiWei/AutoGPTQ (to revision eea67b7e130b92605f5ace33cc93e8b95e9c12a5) to /tmp/pip-req-build-8ae2khnf
  Running command git clone --filter=blob:none --quiet https://github.com/PanQiWei/AutoGPTQ /tmp/pip-req-build-8ae2khnf
  Running command git rev-parse -q --verify 'sha^eea67b7e130b92605f5ace33cc93e8b95e9c12a5'
  Running command git fetch -q https://github.com/PanQiWei/AutoGPTQ eea67b7e130b92605f5ace33cc93e8b95e9c12a5
  Running command git checkout -q eea67b7e130b92605f5ace33cc93e8b95e9c12a5
  Resolved https://github.com/PanQiWei/AutoGPTQ to commit eea67b7e130b92605f5ace33cc93e8b95e9c12a5
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━

In [ ]:
#@title ライブラリの事前準備

from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM

quantized_model_dir = "dahara1/weblab-10b-instruction-sft-GPTQ"
model_basename = "gptq_model-4bit-128g"

tokenizer = AutoTokenizer.from_pretrained(quantized_model_dir)

model = AutoGPTQForCausalLM.from_quantized(
        quantized_model_dir,
        model_basename=model_basename,
        use_safetensors=True,
        device="cuda:0")

In [ ]:
#@title プロンプトの実行
#@markdown AIに実行して貰いたい指示を日本語で設定してください。
prompt_text = "スタジオジブリの作品を５つ教えてください" #@param {type:"string"}

#prompt_text = "greetings!"
#prompt_template = f'以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n\n### Instruce:\n{prompt_text}\n\n### Response:\n\n'
#prompt_template = f'### Instruction: {prompt_text}\n### Response:\n\n'
prompt_template = f'### 指示: {prompt_text}\n### 応答:\n\n'
#prompt_template = f'{prompt_text}'
# Sometimes, Instruction prompt doesn't work, but I can't find reason yet.

tokens = tokenizer(prompt_template, return_tensors="pt").to("cuda:0").input_ids
output = model.generate(input_ids=tokens, max_new_tokens=100, do_sample=True, temperature=0.8, pad_token_id=tokenizer.eos_token_id)
#output = model.generate(input_ids=tokens, max_new_tokens=100, do_sample=True, temperature=0.8)

print(tokenizer.decode(output[0]))

### 指示: スタジオジブリの作品を５つ教えてください
### 応答:

 1.風の谷のナウシカ
 2.となりのトトロ
 3.天空の城ラピュタ
 4.魔女の宅急便
 5.紅の豚
<|endoftext|>
